### AccelerateAI - Data Science Bootcamp
#### Module 01 - Python

***Data analysis is the process by which data becomes understanding, knowledge and insight. ~Hadley Wickham***


In this notebook we will work with Pandas to do some data transformation.
1. Data - Flight, Weather, Planes, Airport 
2. One table operation & grouped summaries
3. Data pipelines
4. Grouped mutate/filter & window functions
5. Joins (two table verbs)

#### 1. Reading data

In [ ]:
import numpy as np
import pandas as pd

#set few options
pd.set_option("display.precision", 3)
pd.set_option('display.min_rows', 8)
pd.set_option('display.width', 2000)

In [ ]:
#Flights data - Every flight departing Houston in 2011 . [227,496 x 14]
flights = pd.read_csv("flights.csv")

# Weather data - Hourly weather data. [8,723 x 14]
weather = pd.read_csv("weather.csv")

#Planes data - Metadata about the planes
planes = pd.read_csv("planes.csv")

#Airports data  - Metadata about airports
airports =  pd.read_csv("airports.csv")

#### Lets have a look at the data and its summary

In [ ]:
flights.sample(5)

In [ ]:
flights.()

In [ ]:
# Convert date string to datetime 
flights['date'] = to_datetime(flights['date'])
flights.info()

In [ ]:
# Finding missing values
flights.isnull().sum()

### What kind of questions can we ask this data?
Find all flights: 
1. Delayed by more than an hour 
2. To SFO in January 
3. That departed between midnight and five am. 
4. Where the arrival delay was more than twice the departure delay

In [ ]:
# Delayed by more than an hour
flights[flights.dep_delay]

In [ ]:
# To SFO in January 
flights.query("dest == 'SFO' and date < '2011-02-01' ")

In [ ]:
# That departed between midnight and five am. 
flights.query("hour >= 0 & hour <= 5")

In [ ]:
# Where the arrival delay was more than twice the departure delay
flights[flights.arr_delay > 2*flights.dep_delay ]

In [ ]:
# Where the arrival delay was more than twice the departure delay
flights.query("arr_delay > 2*dep_delay  arr_delay > 0 & dep_delay > 0")

#### Select 3 columns - date, destination and distance

In [ ]:
flights[['date', 'dest', 'dist']]

#### Ordering Data
Order the flights by departure date and time.
- Which flights were most delayed?
- Which were the longest flights?

In [ ]:
# Order the flights by departure date and time.
flights.sort_values(['date', 'hour', 'minute'])           #ascending by default

In [ ]:
# Wich flights were most delayed?
flights.sort_values('arr_delay', ascending= )

In [ ]:
# Which were the longest flights?
flights.sort_values('dist', ascending=False)

#### Adding new variable
- Compute speed in mph from time (in minutes) and distance (in miles). Which flight flew the fastest?
- Add a new variable that shows how much time was made up or lost in flight.

In [ ]:
# Compute speed in mph from time (in minutes) and distance (in miles). Which flight flew the fastest?
flights['avg_speed'] = flights['dist'] / (flights['time'] / 60)
flights.sort_values('avg_speed', ascending=False) 

In [ ]:
# Flights that caught up the most
flights['caughtup'] = flights['dep_delay'] - flights['arr_delay'] 
flights.sort_values('caughtup', ascending=False)

### Summarizing data
1. Find the average departure delay on each day
2. Which destinations have the highest average delays?
3. Find all the daily flights? Where do they fly to?
4. On average, how do delays (of non-cancelled flights) vary over the course of a day?

In [ ]:
# Average departure delay each day
flights_avg_delay = flights.groupby('date').mean('dep_delay')
flights_avg_delay['dep_delay']

In [ ]:
# Which destinations have the highest average delays?
flights_dest_delay = flights.groupby('dest').agg(mean_dep_delay=('dep_delay',np.mean))
flights_dest_delay.sort_values('mean_dep_delay', ascending=False)

In [ ]:
# Find all the daily flights? Where do they fly to?
flights_summary =  flights.groupby(['carrier', 'flight', 'dest']).count()
flights_summary.query('date ==  ')

In [ ]:
# On average, how do delays (of non-cancelled flights) vary over the course of a day?
flights['fly_time'] = flights.hour + flights.minute/60         
delay_df = flights[flights.cancelled == 0].groupby('fly_time').agg({'dep_delay':['count','']})

In [ ]:
delay_df.head()

In [ ]:
import matplotlib.pyplot as plt

df = delay_df[delay_df['dep_delay']['count'] > 30]       # Filter the small values

plt.figure(figsize=(8,8))
plt.scatter(x= df.index.values, y = df['dep_delay']['mean'], s = 2)
plt.xlabel("Time of Day")
plt.ylabel("Avg Delay (min)")
plt.show()

### Groupwise Calculations
1. Find aircrafts that are delayed on arrival, by more than 2 S.D. of all delays.

In [ ]:
# Calculate avg arr_delay for each "plane" 
summary_df = flights.dropna(axis=0).groupby('plane').agg({'arr_delay':['count','mean']})
summary_df.head()

In [ ]:
#summary_df.describe()  #Some planes had very few flights.

#remove those planes that did not fly much
df = summary_df[summary_df['arr_delay']['count'] > 30]

In [ ]:
# How does the distribution of delay look like
df['arr_delay']['mean'].plot.hist(alpha=0.6)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

df["z_score"] = (df['arr_delay']['mean'] - np.mean(df['arr_delay']['mean']) )/ np.std (df['arr_delay']['mean']) 

In [ ]:
df[df['z_score'] > 2]

### Joining Dataset 
1. What weather conditions are associated with delays leaving in Houston? 

In [ ]:
weather.head()

In [ ]:
#Let's merge the weather data with flight data
weather['date']

merged_df = pd.merge(flights, weather,  how='left', left_on=['date','hour'], right_on = ['date','hour'])
merged_df.head()

In [ ]:
# Create scatter plot between delay and visibility, wind speed and precipitation 
merged_df.plot.scatter(y='dep_delay', x='', s=2)

In [ ]:
merged_df.plot.scatter(y='dep_delay', x='precip', s=2)

In [ ]:
merged_df.plot.scatter(y='dep_delay', x='visibility', s=2)

In [ ]:
planes.head()

### Other Questions:
1. Do older planes cause more delay? 
2. Which manufacture's planes have the best on time performance?
3. Which specific engine types lead to more than twice the average departure delay? 